In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [2]:
data = pd.read_csv('dataset/toxic_comment_train.csv')
data = data.dropna()
X = data['comment_text'].values
y = data['toxic'].values

In [3]:
print(X.shape, y.shape)

(159571,) (159571,)


In [4]:
MAX_FEATURES = 160000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [5]:
vectorizer.adapt(X)

In [13]:
pickle.dump({'config': vectorizer.get_config(),
             'weights': vectorizer.get_weights()}
            , open("vectorzer_toxic.pkl", "wb"))

In [8]:
type(vectorizer)

keras.layers.preprocessing.text_vectorization.TextVectorization

In [82]:
vectorized_text = vectorizer(X)

In [10]:
test = vectorizer("you are the bitch")
test

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 7, 20,  2, ...,  0,  0,  0], dtype=int64)>

In [83]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [84]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [86]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [87]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [88]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          5120032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 256)               33024     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                        

In [89]:
history = model.fit(train, epochs=5, validation_data=val)

Epoch 1/5
6981/6981 [==============================] - 99106s 14s/step - loss: 0.1360 - val_loss: 0.1050
Epoch 2/5
6981/6981 [==============================] - 55721s 8s/step - loss: 0.0908 - val_loss: 0.0739
Epoch 3/5
6981/6981 [==============================] - 56537s 8s/step - loss: 0.0756 - val_loss: 0.0608
Epoch 4/5
6981/6981 [==============================] - 52832s 8s/step - loss: 0.0631 - val_loss: 0.0483
Epoch 5/5
6981/6981 [==============================] - 52736s 8s/step - loss: 0.0513 - val_loss: 0.0397


In [90]:

model.save('toxicity.h5')